# TN DCS CANS Assessment Data Analysis

In [1]:
import pandas as pd
import numpy as np 

In [2]:
pd.set_option('display.max_columns', None) 

<h3 style="color:navy">Import Data</h3>

In [3]:
df_youth = pd.read_csv('../data/bdaic created tables/df_youth.csv', dtype={'PERSON ID': str, 'REMOVAL ZIP CODE': str, 'PLACEMENT ZIP CODE': str}, 
parse_dates=['LOCATION BEGIN DATE', 'LOCATION END DATE'])
df_data = pd.read_csv('../data/bdaic created tables/df_assessment_questions.csv', dtype={'PERSON ID': str}) 

In [4]:
### Summary of Youth Data
print('df_youth shape:\n', df_youth.shape)
print('\ndf_youth columns:\n', df_youth.columns)
print('\ndf_youth dtypes:\n', df_youth.dtypes)

df_youth shape:
 (5684, 11)

df_youth columns:
 Index(['PERSON ID', 'CURRENT AGE', 'COMMITMENT COUNTY', 'RESPONSIBLE COUNTY',
       'GENDER', 'REMOVAL ZIP CODE', 'PLACEMENT ZIP CODE',
       'LOCATION BEGIN DATE', 'LOCATION END DATE', 'CANS', 'LIFESKILL'],
      dtype='object')

df_youth dtypes:
 PERSON ID                      object
CURRENT AGE                     int64
COMMITMENT COUNTY              object
RESPONSIBLE COUNTY             object
GENDER                         object
REMOVAL ZIP CODE               object
PLACEMENT ZIP CODE             object
LOCATION BEGIN DATE    datetime64[ns]
LOCATION END DATE      datetime64[ns]
CANS                          float64
LIFESKILL                     float64
dtype: object


In [5]:
df_youth.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,CANS,LIFESKILL
0,353258,17,Madison,Madison,MALE,37040,37040,2023-06-13,2024-05-25,1.0,0.0
1,706512,17,Sullivan,Sullivan,FEMALE,37642,37642,2024-11-18,NaT,1.0,0.0
2,1348374,17,Warren,Warren,FEMALE,37110,37110,2023-06-02,2023-09-30,1.0,0.0
3,2470086,18,Hamilton,Hamilton,MALE,37218,37218,2024-09-17,NaT,1.0,0.0
4,3965628,21,Davidson,Davidson,MALE,37725,37725,2021-12-22,2022-08-31,1.0,0.0


In [6]:
### Summary of Assessment Data
print('df_data shape:\n', df_data.shape)
print('\ndf_data columns:\n', df_data.columns)
print('\ndf_data dtypes:\n', df_data.dtypes)

df_data shape:
 (555168, 5)

df_data columns:
 Index(['PERSON ID', 'ASSESSMENT TYPE', 'ASSESSMENT INDICATOR', 'SCORE-RESULT',
       'GENERAL RESULT DESCRIPTION'],
      dtype='object')

df_data dtypes:
 PERSON ID                     object
ASSESSMENT TYPE               object
ASSESSMENT INDICATOR          object
SCORE-RESULT                  object
GENERAL RESULT DESCRIPTION    object
dtype: object


In [7]:
# Create summary statistics
total_youth = len(df_youth['PERSON ID'].unique())
cans_youth = len(df_youth[df_youth['CANS'] == 1.0]['PERSON ID'].unique())
lifeskill_youth = len(df_youth[df_youth['LIFESKILL'] == 1.0]['PERSON ID'].unique())
gender_counts = df_youth['GENDER'].value_counts()
age_stats = df_youth['CURRENT AGE'].describe()

# Print summary with formatting
print('Summary of Youth Dataset\n' + '='*25)
print(f'\nTotal Unique Youth: {total_youth:,}')
print(f'Youth with CANS Assessment: {cans_youth:,}')
print(f'Youth with LIFESKILL Assessment: {lifeskill_youth:,}')

print('\nGender Breakdown:')
print('-----------------')
for gender, count in gender_counts.items():
    percentage = (count/len(df_youth))*100
    print(f'{gender}: {count:,} ({percentage:.1f}%)')

print('\nAge Distribution:')
print('----------------')
print(f'Average Age: {age_stats["mean"]:.1f}')
print(f'Minimum Age: {age_stats["min"]:.0f}')
print(f'Maximum Age: {age_stats["max"]:.0f}')
print('\nAge Breakdown:')
age_counts = df_youth['CURRENT AGE'].value_counts().sort_index()
for age, count in age_counts.items():
    percentage = (count/len(df_youth))*100
    print(f'Age {age:.0f}: {count:,} ({percentage:.1f}%)')


Summary of Youth Dataset

Total Unique Youth: 5,684
Youth with CANS Assessment: 5,667
Youth with LIFESKILL Assessment: 374

Gender Breakdown:
-----------------
MALE: 3,342 (58.8%)
FEMALE: 2,342 (41.2%)

Age Distribution:
----------------
Average Age: 18.2
Minimum Age: 16
Maximum Age: 21

Age Breakdown:
Age 16: 108 (1.9%)
Age 17: 1,705 (30.0%)
Age 18: 1,630 (28.7%)
Age 19: 1,295 (22.8%)
Age 20: 897 (15.8%)
Age 21: 49 (0.9%)


In [8]:
### Make some formating adustments to df_data 
# Rename assessment type and indicator columns to proper case
df_data = df_data.rename(columns={
    'ASSESSMENT TYPE': 'Assessment Type',
    'ASSESSMENT INDICATOR': 'Assessment Indicator'
})

df_data.head() 

,PERSON ID,Assessment Type,Assessment Indicator,SCORE-RESULT,GENERAL RESULT DESCRIPTION
0,353258,CANS,Sexual Abuse,0,No current need; no need for action or interv...
1,353258,CANS,Physical Abuse,0,No current need; no need for action or interv...
2,353258,CANS,Emotional Abuse,0,No current need; no need for action or interv...
3,353258,CANS,Neglect,2,Action or intervention is required to ensure ...
4,353258,CANS,Medical Trauma,0,No current need; no need for action or interv...


### Categorize CANS Questions

In [9]:
# Get CANS assessment indicators and counts
cans_indicators = df_data[df_data['Assessment Type'] == 'CANS'].groupby('Assessment Indicator')['PERSON ID'].nunique().reset_index()
cans_indicators.columns = ['Assessment Indicator', 'Number of Youth']

# Calculate percentage of youth who completed CANS
cans_indicators['Percent of Total Youth'] = (cans_indicators['Number of Youth'] / cans_youth * 100).round(1)

# Sort by number of youth descending
cans_indicators = cans_indicators.sort_values('Number of Youth', ascending=False)

print("CANS Assessment Indicators Summary:")
print("==================================")
print(cans_indicators.to_string(index=False))


CANS Assessment Indicators Summary:
                         Assessment Indicator  Number of Youth  Percent of Total Youth
                             Medical/Physical             5667                   100.0
                                     Runaway*             5667                   100.0
                             Family Strengths             5667                   100.0
                                 Fire Setting             5667                   100.0
                  Sexually Reactive Behavior*             5667                   100.0
                           Sexual Development             5667                   100.0
                    Impulsivity/Hyperactivity             5667                   100.0
                            Sexual Aggression             5667                   100.0
                      Intentional Misbehavior             5667                   100.0
           Interpersonal/Social Connectedness             5667                   100.0
       

In [10]:
cans_indicators.sort_values('Assessment Indicator').head(25)

,Assessment Indicator,Number of Youth,Percent of Total Youth
0,Adjustment To Trauma,5641,99.5
1,Anger Control,5667,100.0
2,Anxiety,5667,100.0
3,Arrests,3902,68.9
4,Attachment Difficulties,5667,100.0
5,Attention/Concentration,5667,100.0
6,Avoidance,5641,99.5
7,Bullying Others,5667,100.0
8,CSEM Risk Question 1,2081,36.7
9,CSEM Risk Question 2,3305,58.3


In [11]:
# Create dictionary mapping indicators to categories
indicator_categories = {
    # Mental Health indicators
    'Decision-Making (Judgement)': 'Mental Health',             # Behavioral & Impulse Control
    'Impulsivity/Hyperactivity': 'Mental Health',               # Behavioral & Impulse Control
    'School Behavior': 'Mental Health',                         # Behavioral & Impulse Control
    'Substance Use': 'Mental Health',                           # Behavioral & Impulse Control
    'Spiritual/Religious': 'Mental Health',                     # Cultural, Spiritual, and Engagement Factors
    'Traditions And Rituals': 'Mental Health',                  # Cultural, Spiritual, and Engagement Factors
    'Youth Involvement With Care': 'Mental Health',             # Cultural, Spiritual, and Engagement Factors
    'Anger Control': 'Mental Health',                           # Mood & Emotional Regulation 
    'Anxiety': 'Mental Health',                                 # Mood & Emotional Regulation 
    'Depression': 'Mental Health',                              # Mood & Emotional Regulation 
    'Emotional And/Or Physical Dysregulation': 'Mental Health', # Mood & Emotional Regulation 
    'Numbing': 'Mental Health',                                 # Mood & Emotional Regulation 
    'Medical/Physical': 'Mental Health',                        # Physical & Medical 
    'Psychosis (Thought Disorder)': 'Mental Health',            # Psychotic & Thought Disturbances
    'Danger To Others': 'Mental Health',                        # Risk & Safety
    'Intentional Misbehavior': 'Mental Health',                 # Risk & Safety  
    'Non-Suicidal Self-Injurious Behavior': 'Mental Health',    # Risk & Safety
    'Other Self-Harm (Recklessness)': 'Mental Health',          # Risk & Safety
    'Suicide Risk': 'Mental Health',                            # Risk & Safety 
    'Sexual Aggression': 'Mental Health',                       # Sexual & Developmental Concerns
    'Sexual Development': 'Mental Health',                      # Sexual & Developmental Concerns
    'Adjustment To Trauma': 'Mental Health',                    # Trauma & Stress-Related
    'Cultural Stress': 'Mental Health',                         # Trauma & Stress-Related
    'Emotional Abuse': 'Mental Health',                         # Trauma & Stress-Related
    'Environmental Influences': 'Mental Health',                # Trauma & Stress-Related
    'Medical Trauma': 'Mental Health',                          # Trauma & Stress-Related
    'Sexual Abuse': 'Mental Health',                            # Trauma & Stress-Related
    
    
    # Supportive Adult Relationships indicators
    'Attachment Difficulties': 'Supportive Adult Relationships',
    'Community Life': 'Supportive Adult Relationships',
    'Family Functioning': 'Supportive Adult Relationships',
    'Family Strengths': 'Supportive Adult Relationships',
    'Interpersonal/Social Connectedness': 'Supportive Adult Relationships',
    'Natural Supports': 'Supportive Adult Relationships',
    'Neglect': 'Supportive Adult Relationships',
    'Oppositional(Non-compliance with Authority)': 'Supportive Adult Relationships',
    'Relationship Permanence': 'Supportive Adult Relationships',
    'Social Functioning': 'Supportive Adult Relationships',
    
    # Housing Instability indicators
    'Independent Living Skills': 'Housing',
    'Living Situation': 'Housing Instability', 
    'Physical Abuse': 'Housing Instability',
    'Runaway*': 'Housing Instability',
    'Youth Residential Stability': 'Housing Instability'
}

# Add category column
cans_indicators['Pillar'] = cans_indicators['Assessment Indicator'].map(indicator_categories)

# Display categorized indicators
print("\nCategorized CANS Assessment Indicators:")
print("======================================")
for category in ['Mental Health', 'Supportive Adult Relationships', 'Housing Instability']:
    print(f"\n{category}:")
    print(cans_indicators[cans_indicators['Pillar'] == category][['Assessment Indicator', 'Number of Youth']].to_string(index=False))



Categorized CANS Assessment Indicators:

Mental Health:
                   Assessment Indicator  Number of Youth
                       Medical/Physical             5667
                     Sexual Development             5667
              Impulsivity/Hyperactivity             5667
                      Sexual Aggression             5667
                Intentional Misbehavior             5667
                           Sexual Abuse             5667
                        School Behavior             5667
                         Medical Trauma             5667
                          Anger Control             5667
   Non-Suicidal Self-Injurious Behavior             5667
         Other Self-Harm (Recklessness)             5667
                        Emotional Abuse             5667
                                Anxiety             5667
                 Traditions And Rituals             5667
                           Suicide Risk             5667
                          Subst

In [12]:
### Filter CANS indicators to pillar-related indicators only 
pillar_indicators = cans_indicators[cans_indicators['Pillar'].notna()]

print(pillar_indicators.shape)
pillar_indicators.head()

(42, 4)


,Assessment Indicator,Number of Youth,Percent of Total Youth,Pillar
50,Medical/Physical,5667,100.0,Mental Health
69,Runaway*,5667,100.0,Housing Instability
33,Family Strengths,5667,100.0,Supportive Adult Relationships
78,Sexual Development,5667,100.0,Mental Health
37,Impulsivity/Hyperactivity,5667,100.0,Mental Health


In [13]:
   Behavioral_impulse_control ={ 'Decision-Making (Judgement)': 'Mental Health',             # Behavioral & Impulse Control
    'Impulsivity/Hyperactivity': 'Mental Health',               # Behavioral & Impulse Control
    'School Behavior': 'Mental Health',                         # Behavioral & Impulse Control
    'Substance Use': 'Mental Health',  # Behavioral & Impulse Control
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'

   }
   Cultural_spiritual_engagement ={ 'Spiritual/Religious': 'Mental Health',                     # Cultural, Spiritual, and Engagement Factors
    'Traditions And Rituals': 'Mental Health',                  # Cultural, Spiritual, and Engagement Factors
    'Youth Involvement With Care': 'Mental Health' ,            # Cultural, Spiritual, and Engagement Factors
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
   }
   Mood_emotional_regulation ={ 'Anger Control': 'Mental Health',                           # Mood & Emotional Regulation 
    'Anxiety': 'Mental Health',                                 # Mood & Emotional Regulation 
    'Depression': 'Mental Health',                              # Mood & Emotional Regulation 
    'Emotional And/Or Physical Dysregulation': 'Mental Health', # Mood & Emotional Regulation 
    'Numbing': 'Mental Health' ,                               # Mood & Emotional Regulation 
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
   }
   Physical_medical ={ 'Medical/Physical': 'Mental Health',                        # Physical & Medical 
   'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
   }
   Psychotic_thought_disturbances ={ 'Psychosis (Thought Disorder)': 'Mental Health'  ,     # Psychotic & Thought Disturbances
   'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
   }

Risk_safety ={ 'Danger To Others': 'Mental Health',       # Risk & Safety
    'Intentional Misbehavior': 'Mental Health',                 # Risk & Safety  
    'Non-Suicidal Self-Injurious Behavior': 'Mental Health',    # Risk & Safety
    'Other Self-Harm (Recklessness)': 'Mental Health',          # Risk & Safety
    'Suicide Risk': 'Mental Health',                            # Risk & Safety 
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
Sexual_developmental_concerns ={ 'Sexual Aggression': 'Mental Health',                       # Sexual & Developmental Concerns
    'Sexual Development': 'Mental Health',                      # Sexual & Developmental Concerns
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' :  'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' :  'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
Trauma_stress_related ={ 'Adjustment To Trauma': 'Mental Health',                    # Trauma & Stress-Related
    'Cultural Stress': 'Mental Health',                         # Trauma & Stress-Related
    'Emotional Abuse': 'Mental Health',                         # Trauma & Stress-Related
    'Environmental Influences': 'Mental Health',                # Trauma & Stress-Related
    'Medical Trauma': 'Mental Health',                          # Trauma & Stress-Related
    'Sexual Abuse': 'Mental Health'  ,                          # Trauma & Stress-Related
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }

In [14]:
    # Supportive Adult Relationships indicators
    Supportive_adult = {
    'Attachment Difficulties': 'Supportive Adult Relationships',
    'Community Life': 'Supportive Adult Relationships',
    'Family Functioning': 'Supportive Adult Relationships',
    'Family Strengths': 'Supportive Adult Relationships',
    'Interpersonal/Social Connectedness': 'Supportive Adult Relationships',
    'Natural Supports': 'Supportive Adult Relationships',
    'Neglect': 'Supportive Adult Relationships',
    'Oppositional(Non-compliance with Authority)': 'Supportive Adult Relationships',
    'Relationship Permanence': 'Supportive Adult Relationships',
    'Social Functioning': 'Supportive Adult Relationships',
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }
    
    # Housing Instability indicators
    Housing_instability = {
    'Independent Living Skills': 'Housing',
    'Living Situation': 'Housing Instability', 
    'Physical Abuse': 'Housing Instability',
    'Runaway*': 'Housing Instability',
    'Youth Residential Stability': 'Housing Instability',
    'PERSON ID' :'PERSON ID',                        
    'GENDER' : 'GENDER',
   'PLACEMENT ZIP CODE' : 'PLACEMENT ZIP CODE',
   'RESPONSIBLE COUNTY' : 'RESPONSIBLE COUNTY',
   'Region' : 'Region'
    }

<h3 style="color:navy">Filter CANS indicators to pillar-related indicators only</h3>

In [15]:
df_pillar = df_data.merge(pillar_indicators[['Assessment Indicator', 'Pillar']], how='inner', on='Assessment Indicator') 

print(df_pillar.shape)
df_pillar.head()


(232744, 6)


,PERSON ID,Assessment Type,Assessment Indicator,SCORE-RESULT,GENERAL RESULT DESCRIPTION,Pillar
0,353258,CANS,Sexual Abuse,0,No current need; no need for action or interv...,Mental Health
1,706512,CANS,Sexual Abuse,1,"Identified need requires monitoring, watchful...",Mental Health
2,1348374,CANS,Sexual Abuse,2,Action or intervention is required to ensure ...,Mental Health
3,2470086,CANS,Sexual Abuse,0,No current need; no need for action or interv...,Mental Health
4,3965628,CANS,Sexual Abuse,0,No current need; no need for action or interv...,Mental Health


In [16]:
df_pillar_pivot = df_pillar.pivot_table(index='PERSON ID', columns='Assessment Indicator', values='SCORE-RESULT', aggfunc='first')

print(df_pillar_pivot.shape)
df_pillar_pivot.head()




(5667, 42)


Assessment Indicator,Adjustment To Trauma,Anger Control,Anxiety,Attachment Difficulties,Community Life,Cultural Stress,Danger To Others,Decision-Making (Judgement),Depression,Emotional Abuse,Emotional And/Or Physical Dysregulation,Environmental Influences,Family Functioning,Family Strengths,Impulsivity/Hyperactivity,Independent Living Skills,Intentional Misbehavior,Interpersonal/Social Connectedness,Living Situation,Medical Trauma,Medical/Physical,Natural Supports,Neglect,Non-Suicidal Self-Injurious Behavior,Numbing,Oppositional(Non-compliance with Authority),Other Self-Harm (Recklessness),Physical Abuse,Psychosis (Thought Disorder),Relationship Permanence,Runaway*,School Behavior,Sexual Abuse,Sexual Aggression,Sexual Development,Social Functioning,Spiritual/Religious,Substance Use,Suicide Risk,Traditions And Rituals,Youth Involvement With Care,Youth Residential Stability
PERSON ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10003244,0,0,0,0,3,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0
10004996,1,1,2,1,2,0,1,1,2,1,1,0,1,2,2,1,1,2,1,0,1,3,2,1,1,1,1,2,0,2,0,1,1,0,1,2,3,0,1,0,0,1
100079976,1,1,2,1,3,0,1,2,0,0,2,0,1,3,2,0,1,2,1,0,0,3,0,1,1,1,1,0,0,2,1,1,2,0,1,2,3,0,0,0,1,0
10010134,2,0,1,0,3,0,0,1,1,1,0,0,2,2,1,0,0,0,0,0,0,1,3,1,0,1,0,1,0,2,0,0,0,0,0,0,3,1,1,0,0,1
100106936,1,1,1,0,2,0,0,1,1,0,1,1,1,2,1,1,0,1,0,0,0,1,3,0,0,1,1,0,0,1,0,0,0,0,2,1,2,1,1,0,0,1


<h3 style="color:navy">Merge df_youth with df_pillar_pivot</h3>

In [17]:
df_cans = df_youth.merge(df_pillar_pivot, how='inner', on='PERSON ID').drop(columns=['CANS', 'LIFESKILL'])

print(df_cans.shape)
df_cans.head()

(5667, 51)


,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Adjustment To Trauma,Anger Control,Anxiety,Attachment Difficulties,Community Life,Cultural Stress,Danger To Others,Decision-Making (Judgement),Depression,Emotional Abuse,Emotional And/Or Physical Dysregulation,Environmental Influences,Family Functioning,Family Strengths,Impulsivity/Hyperactivity,Independent Living Skills,Intentional Misbehavior,Interpersonal/Social Connectedness,Living Situation,Medical Trauma,Medical/Physical,Natural Supports,Neglect,Non-Suicidal Self-Injurious Behavior,Numbing,Oppositional(Non-compliance with Authority),Other Self-Harm (Recklessness),Physical Abuse,Psychosis (Thought Disorder),Relationship Permanence,Runaway*,School Behavior,Sexual Abuse,Sexual Aggression,Sexual Development,Social Functioning,Spiritual/Religious,Substance Use,Suicide Risk,Traditions And Rituals,Youth Involvement With Care,Youth Residential Stability
0,353258,17,Madison,Madison,MALE,37040,37040,2023-06-13,2024-05-25,1,0,1,0,2,0,0,1,1,0,1,NaN,1,1,0,0,1,1,1,0,0,2,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,706512,17,Sullivan,Sullivan,FEMALE,37642,37642,2024-11-18,NaT,1,0,0,1,1,0,0,1,0,0,0,NaN,1,2,0,0,0,1,0,1,0,3,3,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0
2,1348374,17,Warren,Warren,FEMALE,37110,37110,2023-06-02,2023-09-30,1,1,1,1,3,0,0,1,1,0,0,0,1,0,1,1,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,2,0,0,1,1,1,1,0,0,0
3,2470086,18,Hamilton,Hamilton,MALE,37218,37218,2024-09-17,NaT,0,2,2,0,2,0,2,2,2,0,0,0,1,0,0,2,0,2,1,0,1,3,1,0,0,0,1,0,1,2,1,1,0,0,0,2,3,2,1,0,1,2
4,3965628,21,Davidson,Davidson,MALE,37725,37725,2021-12-22,2022-08-31,2,0,1,1,1,0,2,2,1,0,0,2,0,0,2,0,1,0,0,1,2,0,1,0,0,3,3,0,0,0,0,1,0,0,0,1,0,2,0,0,1,0


In [18]:
# Convert indicator_category keys to list
indicator_cols = list(indicator_categories.keys())


### Reorder columns
# Get first 9 columns
first_cols = df_cans.columns[:9]

# Create ordered column list 
ordered_cols = list(first_cols) + indicator_cols

# Reorder dataframe, keeping only columns that exist
df_cans = df_cans[ordered_cols]

print(df_cans.shape)
df_cans.head()


(5667, 51)


,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,Depression,Emotional And/Or Physical Dysregulation,Numbing,Medical/Physical,Psychosis (Thought Disorder),Danger To Others,Intentional Misbehavior,Non-Suicidal Self-Injurious Behavior,Other Self-Harm (Recklessness),Suicide Risk,Sexual Aggression,Sexual Development,Adjustment To Trauma,Cultural Stress,Emotional Abuse,Environmental Influences,Medical Trauma,Sexual Abuse,Attachment Difficulties,Community Life,Family Functioning,Family Strengths,Interpersonal/Social Connectedness,Natural Supports,Neglect,Oppositional(Non-compliance with Authority),Relationship Permanence,Social Functioning,Independent Living Skills,Living Situation,Physical Abuse,Runaway*,Youth Residential Stability
0,353258,17,Madison,Madison,MALE,37040,37040,2023-06-13,2024-05-25,1,0,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,NaN,0,0,0,2,1,1,1,2,2,0,1,0,0,1,0,0,0
1,706512,17,Sullivan,Sullivan,FEMALE,37642,37642,2024-11-18,NaT,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,NaN,1,1,1,1,1,2,1,3,3,0,1,0,0,0,0,0,0
2,1348374,17,Warren,Warren,FEMALE,37110,37110,2023-06-02,2023-09-30,1,1,1,1,1,0,0,1,1,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,2,1,3,1,0,1,1,0,1,0,1,1,0,0,0,0
3,2470086,18,Hamilton,Hamilton,MALE,37218,37218,2024-09-17,NaT,2,0,1,2,3,0,1,2,2,2,0,0,1,1,2,0,0,1,1,0,0,0,0,0,0,0,0,0,2,1,0,2,3,1,0,2,2,2,1,0,1,2
4,3965628,21,Davidson,Davidson,MALE,37725,37725,2021-12-22,2022-08-31,2,2,1,2,0,0,1,0,1,1,0,0,2,0,2,1,0,3,0,0,0,2,0,0,2,1,0,1,1,0,0,0,0,1,3,0,1,0,0,0,0,0


In [19]:
tn_counties_by_region = {
    "East": [
        "Anderson", "Bledsoe", "Blount", "Bradley", "Campbell", "Carter",
        "Claiborne", "Cocke", "Cumberland", "Grainger", "Greene", "Hamblen",
        "Hamilton", "Hancock", "Hawkins", "Jefferson", "Johnson", "Knox",
        "Loudon", "Marion", "McMinn", "Meigs", "Monroe", "Morgan", "Polk",
        "Rhea", "Roane", "Scott", "Sevier", "Sullivan", "Unicoi", "Union",
        "Washington"
    ],
    "Middle": [
        "Bedford", "Cannon", "Cheatham", "Clay", "Coffee", "Davidson",
        "DeKalb", "Dickson", "Fentress", "Franklin", "Giles", "Grundy",
        "Hickman", "Houston", "Humphreys", "Jackson", "Lawrence", "Lewis",
        "Lincoln", "Macon", "Marshall", "Maury", "Moore", "Overton", "Perry",
        "Pickett", "Putnam", "Robertson", "Rutherford", "Sequatchie", "Smith",
        "Stewart", "Sumner", "Trousdale", "Van Buren", "Warren", "Wayne",
        "White", "Williamson", "Wilson"
    ],
    "West": [
        "Benton", "Carroll", "Chester", "Crockett", "Decatur", "Dyer",
        "Fayette", "Gibson", "Hardeman", "Hardin", "Haywood", "Henderson",
        "Henry", "Lake", "Lauderdale", "Madison", "McNairy", "Obion",
        "Shelby", "Tipton", "Weakley"
    ]
}


In [20]:
county_to_region = {
    county: region
    for region, counties in tn_counties_by_region.items()
    for county in counties
}

df_cans['Region'] = (
    df_cans['RESPONSIBLE COUNTY'].astype(str).str.strip().map(county_to_region)
)
df_cans['Region'] = df_cans['Region'].fillna('Unknown')

In [21]:
df_cans.head()

,PERSON ID,CURRENT AGE,COMMITMENT COUNTY,RESPONSIBLE COUNTY,GENDER,REMOVAL ZIP CODE,PLACEMENT ZIP CODE,LOCATION BEGIN DATE,LOCATION END DATE,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,Depression,Emotional And/Or Physical Dysregulation,Numbing,Medical/Physical,Psychosis (Thought Disorder),Danger To Others,Intentional Misbehavior,Non-Suicidal Self-Injurious Behavior,Other Self-Harm (Recklessness),Suicide Risk,Sexual Aggression,Sexual Development,Adjustment To Trauma,Cultural Stress,Emotional Abuse,Environmental Influences,Medical Trauma,Sexual Abuse,Attachment Difficulties,Community Life,Family Functioning,Family Strengths,Interpersonal/Social Connectedness,Natural Supports,Neglect,Oppositional(Non-compliance with Authority),Relationship Permanence,Social Functioning,Independent Living Skills,Living Situation,Physical Abuse,Runaway*,Youth Residential Stability,Region
0,353258,17,Madison,Madison,MALE,37040,37040,2023-06-13,2024-05-25,1,0,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,NaN,0,0,0,2,1,1,1,2,2,0,1,0,0,1,0,0,0,West
1,706512,17,Sullivan,Sullivan,FEMALE,37642,37642,2024-11-18,NaT,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,NaN,1,1,1,1,1,2,1,3,3,0,1,0,0,0,0,0,0,East
2,1348374,17,Warren,Warren,FEMALE,37110,37110,2023-06-02,2023-09-30,1,1,1,1,1,0,0,1,1,1,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,2,1,3,1,0,1,1,0,1,0,1,1,0,0,0,0,Middle
3,2470086,18,Hamilton,Hamilton,MALE,37218,37218,2024-09-17,NaT,2,0,1,2,3,0,1,2,2,2,0,0,1,1,2,0,0,1,1,0,0,0,0,0,0,0,0,0,2,1,0,2,3,1,0,2,2,2,1,0,1,2,East
4,3965628,21,Davidson,Davidson,MALE,37725,37725,2021-12-22,2022-08-31,2,2,1,2,0,0,1,0,1,1,0,0,2,0,2,1,0,3,0,0,0,2,0,0,2,1,0,1,1,0,0,0,0,1,3,0,1,0,0,0,0,0,Middle


## Risk greater than 2 for all

### Behvioral Impulse

In [67]:
rename_map = Behavioral_impulse_control
cols_to_keep = list(rename_map.keys())
df_bic = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=rename_map).copy()

In [79]:
df_bic.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,1,0,0,1,West
1,706512,Sullivan,FEMALE,37642,1,0,0,0,East
2,1348374,Warren,FEMALE,37110,1,1,1,1,Middle
3,2470086,Hamilton,MALE,37218,2,0,1,2,East
4,3965628,Davidson,MALE,37725,2,2,1,2,Middle


In [81]:
# --- Select Mental Health Columns ---
mh_cols = df_bic.filter(regex=r'^Mental Health$').columns
nums_mh = df_bic[mh_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mh = nums_mh.ge(2).any(axis=1)
df_bic_2 = df_bic[mask_mh]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mh_cols_all = df_bic.filter(regex=r'^Mental Health$').columns
nums = df_bic[mh_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_bic = df_bic[base_cols].copy()
new_df_bic['Flag_MentalHealthIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_bic['Flag_MentalHealthIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_MentalHealthIssue = 1")

# --- Percentage of Flag_MentalHealthIssue = 1 by Gender ---
gender_pct = new_df_bic.groupby('GENDER')['Flag_MentalHealthIssue'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_bic.groupby('GENDER')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage of Flag_MentalHealthIssue = 1 by Region ---
region_summary = new_df_bic.groupby('Region')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)

60.93% of rows have Flag_MentalHealthIssue = 1

% by Gender:
 GENDER
FEMALE    50.836551
MALE      67.985612
Name: Flag_MentalHealthIssue, dtype: float64

Gender Summary:
          sum  count    percent
GENDER                        
FEMALE  1185   2331  50.836551
MALE    2268   3336  67.985612

Region Summary:
           sum  count    percent
Region                         
East     1285   2172  59.162063
Middle   1228   2037  60.284732
Unknown   117    255  45.882353
West      823   1203  68.412303


### Cultural/Spiritual bucket

In [24]:
cols_to_keep = list(Cultural_spiritual_engagement .keys())
df_cse = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Cultural_spiritual_engagement).copy()

In [25]:
df_cse.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,0,0,0,West
1,706512,Sullivan,FEMALE,37642,1,0,0,East
2,1348374,Warren,FEMALE,37110,1,0,0,Middle
3,2470086,Hamilton,MALE,37218,3,0,1,East
4,3965628,Davidson,MALE,37725,0,0,1,Middle


In [82]:
# --- Select Mental Health Columns ---
mh_cols = df_cse.filter(regex=r'^Mental Health$').columns
nums_mh = df_cse[mh_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mh = nums_mh.ge(2).any(axis=1)
df_cse_2 = df_cse[mask_mh]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mh_cols_all = df_cse.filter(regex=r'^Mental Health$').columns
nums = df_cse[mh_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_cse = df_cse[base_cols].copy()
new_df_cse['Flag_MentalHealthIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_cse['Flag_MentalHealthIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_MentalHealthIssue = 1")

# --- Percentage of Flag_MentalHealthIssue = 1 by Gender ---
gender_pct = new_df_cse.groupby('GENDER')['Flag_MentalHealthIssue'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_cse.groupby('GENDER')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage of Flag_MentalHealthIssue = 1 by Region ---
region_summary = new_df_cse.groupby('Region')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)


71.87% of rows have Flag_MentalHealthIssue = 1

% by Gender:
 GENDER
FEMALE    70.313170
MALE      72.961631
Name: Flag_MentalHealthIssue, dtype: float64

Gender Summary:
          sum  count    percent
GENDER                        
FEMALE  1639   2331  70.313170
MALE    2434   3336  72.961631

Region Summary:
           sum  count    percent
Region                         
East     1699   2172  78.222836
Middle   1447   2037  71.035837
Unknown   148    255  58.039216
West      779   1203  64.754780


### Mood/emotional regulation

In [26]:
cols_to_keep = list(Mood_emotional_regulation.keys())
df_mer = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Mood_emotional_regulation).copy()

In [27]:
df_mer.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,0,1,1,1,0,West
1,706512,Sullivan,FEMALE,37642,0,0,0,0,0,East
2,1348374,Warren,FEMALE,37110,1,1,1,0,0,Middle
3,2470086,Hamilton,MALE,37218,2,2,2,0,0,East
4,3965628,Davidson,MALE,37725,0,1,1,0,0,Middle


In [83]:
# --- Select Mental Health Columns ---
mh_cols = df_mer.filter(regex=r'^Mental Health$').columns
nums_mh = df_mer[mh_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mh = nums_mh.ge(2).any(axis=1)
df_mer_2 = df_mer[mask_mh]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mh_cols_all = df_mer.filter(regex=r'^Mental Health$').columns
nums = df_mer[mh_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_mer = df_mer[base_cols].copy()
new_df_mer['Flag_MentalHealthIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_mer['Flag_MentalHealthIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_MentalHealthIssue = 1")

# --- Percentage of Flag_MentalHealthIssue = 1 by Gender ---
gender_pct = new_df_mer.groupby('GENDER')['Flag_MentalHealthIssue'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_mer.groupby('GENDER')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage of Flag_MentalHealthIssue = 1 by Region ---
region_summary = new_df_mer.groupby('Region')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)

54.40% of rows have Flag_MentalHealthIssue = 1

% by Gender:
 GENDER
FEMALE    60.06006
MALE      50.44964
Name: Flag_MentalHealthIssue, dtype: float64

Gender Summary:
          sum  count   percent
GENDER                       
FEMALE  1400   2331  60.06006
MALE    1683   3336  50.44964

Region Summary:
           sum  count    percent
Region                         
East     1360   2172  62.615101
Middle    996   2037  48.895434
Unknown   105    255  41.176471
West      622   1203  51.704073


### Physical_medical


In [28]:
cols_to_keep = list(Physical_medical.keys())
df_pm = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Physical_medical).copy()

In [29]:
df_pm.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Region
0,353258,Madison,MALE,37040,0,West
1,706512,Sullivan,FEMALE,37642,0,East
2,1348374,Warren,FEMALE,37110,0,Middle
3,2470086,Hamilton,MALE,37218,1,East
4,3965628,Davidson,MALE,37725,2,Middle


In [84]:
# --- Select Mental Health Columns ---
mh_cols = df_pm.filter(regex=r'^Mental Health$').columns
nums_mh = df_pm[mh_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mh = nums_mh.ge(2).any(axis=1)
df_pm_2 = df_pm[mask_mh]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mh_cols_all = df_pm.filter(regex=r'^Mental Health$').columns
nums = df_pm[mh_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_pm = df_pm[base_cols].copy()
new_df_pm['Flag_MentalHealthIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_pm['Flag_MentalHealthIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_MentalHealthIssue = 1")

# --- Percentage of Flag_MentalHealthIssue = 1 by Gender ---
gender_pct = new_df_pm.groupby('GENDER')['Flag_MentalHealthIssue'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_pm.groupby('GENDER')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage of Flag_MentalHealthIssue = 1 by Region ---
region_summary = new_df_pm.groupby('Region')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)


6.49% of rows have Flag_MentalHealthIssue = 1

% by Gender:
 GENDER
FEMALE    6.992707
MALE      6.145084
Name: Flag_MentalHealthIssue, dtype: float64

Gender Summary:
         sum  count   percent
GENDER                      
FEMALE  163   2331  6.992707
MALE    205   3336  6.145084

Region Summary:
          sum  count   percent
Region                       
East     160   2172  7.366483
Middle   117   2037  5.743741
Unknown   12    255  4.705882
West      79   1203  6.566916


### Psychotic_thought_disturbances


In [30]:

cols_to_keep = list(Psychotic_thought_disturbances.keys())
df_ptd = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Psychotic_thought_disturbances).copy()

In [31]:
df_ptd.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Region
0,353258,Madison,MALE,37040,0,West
1,706512,Sullivan,FEMALE,37642,0,East
2,1348374,Warren,FEMALE,37110,0,Middle
3,2470086,Hamilton,MALE,37218,1,East
4,3965628,Davidson,MALE,37725,0,Middle


In [85]:
# --- Select Mental Health Columns ---
mh_cols = df_ptd.filter(regex=r'^Mental Health$').columns
nums_mh = df_ptd[mh_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mh = nums_mh.ge(2).any(axis=1)
df_ptd_2 = df_ptd[mask_mh]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mh_cols_all = df_ptd.filter(regex=r'^Mental Health$').columns
nums = df_ptd[mh_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_ptd = df_ptd[base_cols].copy()
new_df_ptd['Flag_MentalHealthIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_ptd['Flag_MentalHealthIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_MentalHealthIssue = 1")

# --- Percentage of Flag_MentalHealthIssue = 1 by Gender ---
gender_pct = new_df_ptd.groupby('GENDER')['Flag_MentalHealthIssue'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_ptd.groupby('GENDER')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage of Flag_MentalHealthIssue = 1 by Region ---
region_summary = new_df_ptd.groupby('Region')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)


2.28% of rows have Flag_MentalHealthIssue = 1

% by Gender:
 GENDER
FEMALE    2.359502
MALE      2.218225
Name: Flag_MentalHealthIssue, dtype: float64

Gender Summary:
         sum  count   percent
GENDER                      
FEMALE   55   2331  2.359502
MALE     74   3336  2.218225

Region Summary:
          sum  count   percent
Region                       
East      44   2172  2.025783
Middle    39   2037  1.914580
Unknown   10    255  3.921569
West      36   1203  2.992519


### Risk_safety


In [32]:
cols_to_keep = list(Risk_safety.keys())
df_rs = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Risk_safety).copy()

In [33]:
df_rs.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,0,1,0,0,0,West
1,706512,Sullivan,FEMALE,37642,0,0,0,0,0,East
2,1348374,Warren,FEMALE,37110,0,1,1,0,1,Middle
3,2470086,Hamilton,MALE,37218,2,0,0,1,1,East
4,3965628,Davidson,MALE,37725,2,1,0,3,0,Middle


In [86]:
# --- Select Mental Health Columns ---
mh_cols = df_rs.filter(regex=r'^Mental Health$').columns
nums_mh = df_rs[mh_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mh = nums_mh.ge(2).any(axis=1)
df_rs_2 = df_rs[mask_mh]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mh_cols_all = df_rs.filter(regex=r'^Mental Health$').columns
nums = df_rs[mh_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_rs = df_rs[base_cols].copy()
new_df_rs['Flag_MentalHealthIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_rs['Flag_MentalHealthIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_MentalHealthIssue = 1")

# --- Percentage of Flag_MentalHealthIssue = 1 by Gender ---
gender_pct = new_df_rs.groupby('GENDER')['Flag_MentalHealthIssue'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_rs.groupby('GENDER')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage of Flag_MentalHealthIssue = 1 by Region ---
region_summary = new_df_rs.groupby('Region')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)


28.60% of rows have Flag_MentalHealthIssue = 1

% by Gender:
 GENDER
FEMALE    24.324324
MALE      31.594724
Name: Flag_MentalHealthIssue, dtype: float64

Gender Summary:
          sum  count    percent
GENDER                        
FEMALE   567   2331  24.324324
MALE    1054   3336  31.594724

Region Summary:
          sum  count    percent
Region                        
East     528   2172  24.309392
Middle   562   2037  27.589593
Unknown   61    255  23.921569
West     470   1203  39.068994


### Sexual_developmental_concerns

In [34]:
cols_to_keep = list(Sexual_developmental_concerns.keys())
df_sdc = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Sexual_developmental_concerns).copy()

In [35]:
df_sdc.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,0,0,West
1,706512,Sullivan,FEMALE,37642,0,1,East
2,1348374,Warren,FEMALE,37110,0,0,Middle
3,2470086,Hamilton,MALE,37218,0,0,East
4,3965628,Davidson,MALE,37725,0,0,Middle


In [87]:
# --- Select Mental Health Columns ---
mh_cols = df_sdc.filter(regex=r'^Mental Health$').columns
nums_mh = df_sdc[mh_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mh = nums_mh.ge(2).any(axis=1)
df_sdc_2 = df_sdc[mask_mh]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mh_cols_all = df_sdc.filter(regex=r'^Mental Health$').columns
nums = df_sdc[mh_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_sdc = df_sdc[base_cols].copy()
new_df_sdc['Flag_MentalHealthIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_sdc['Flag_MentalHealthIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_MentalHealthIssue = 1")

# --- Percentage of Flag_MentalHealthIssue = 1 by Gender ---
gender_pct = new_df_sdc.groupby('GENDER')['Flag_MentalHealthIssue'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_sdc.groupby('GENDER')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage of Flag_MentalHealthIssue = 1 by Region ---
region_summary = new_df_sdc.groupby('Region')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)


8.51% of rows have Flag_MentalHealthIssue = 1

% by Gender:
 GENDER
FEMALE    9.652510
MALE      7.703837
Name: Flag_MentalHealthIssue, dtype: float64

Gender Summary:
         sum  count   percent
GENDER                      
FEMALE  225   2331  9.652510
MALE    257   3336  7.703837

Region Summary:
          sum  count   percent
Region                       
East     199   2172  9.162063
Middle   188   2037  9.229259
Unknown   17    255  6.666667
West      78   1203  6.483791


### Trauma_stress_related

In [36]:
cols_to_keep = list(Trauma_stress_related.keys())
df_tsr = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Trauma_stress_related).copy()

In [37]:
df_tsr.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Mental Health,Mental Health,Mental Health,Mental Health,Mental Health,Mental Health,Region
0,353258,Madison,MALE,37040,1,0,0,NaN,0,0,West
1,706512,Sullivan,FEMALE,37642,1,0,0,NaN,1,1,East
2,1348374,Warren,FEMALE,37110,1,0,0,0,0,2,Middle
3,2470086,Hamilton,MALE,37218,0,0,0,0,0,0,East
4,3965628,Davidson,MALE,37725,2,0,0,2,1,0,Middle


In [88]:
# --- Select Mental Health Columns ---
mh_cols = df_tsr.filter(regex=r'^Mental Health$').columns
nums_mh = df_tsr[mh_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_mh = nums_mh.ge(2).any(axis=1)
df_tsr_2 = df_tsr[mask_mh]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
mh_cols_all = df_tsr.filter(regex=r'^Mental Health$').columns
nums = df_tsr[mh_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_tsr = df_tsr[base_cols].copy()
new_df_tsr['Flag_MentalHealthIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_tsr['Flag_MentalHealthIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_MentalHealthIssue = 1")

# --- Percentage of Flag_MentalHealthIssue = 1 by Gender ---
gender_pct = new_df_tsr.groupby('GENDER')['Flag_MentalHealthIssue'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_tsr.groupby('GENDER')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage of Flag_MentalHealthIssue = 1 by Region ---
region_summary = new_df_tsr.groupby('Region')['Flag_MentalHealthIssue'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)


52.20% of rows have Flag_MentalHealthIssue = 1

% by Gender:
 GENDER
FEMALE    62.462462
MALE      45.023981
Name: Flag_MentalHealthIssue, dtype: float64

Gender Summary:
          sum  count    percent
GENDER                        
FEMALE  1456   2331  62.462462
MALE    1502   3336  45.023981

Region Summary:
           sum  count    percent
Region                         
East     1283   2172  59.069982
Middle   1018   2037  49.975454
Unknown    94    255  36.862745
West      563   1203  46.799667


### Supportive_adult

In [38]:
cols_to_keep = list(Supportive_adult.keys())
df_sa = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Supportive_adult).copy()

In [39]:
df_sa.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Supportive Adult Relationships,Region
0,353258,Madison,MALE,37040,0,2,1,1,1,2,2,0,1,0,West
1,706512,Sullivan,FEMALE,37642,1,1,1,2,1,3,3,0,1,0,East
2,1348374,Warren,FEMALE,37110,1,3,1,0,1,1,0,1,0,1,Middle
3,2470086,Hamilton,MALE,37218,0,2,1,0,2,3,1,0,2,2,East
4,3965628,Davidson,MALE,37725,1,1,0,0,0,0,1,3,0,1,Middle


In [89]:
# --- Select Supportive Adult Relationships columns ---
sar_cols = df_sa.filter(regex=r'^Supportive Adult Relationships$').columns
nums_sar = df_sa[sar_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask_sar = nums_sar.ge(2).any(axis=1)
df_sa_2 = df_sa[mask_sar]

# --- Build New Dataset with PERSON ID, ZIP, REGION, GENDER ---
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
sar_cols_all = df_sa.filter(regex=r'^Supportive Adult Relationships$').columns
nums = df_sa[sar_cols_all].apply(pd.to_numeric, errors='coerce')
mask = nums.ge(2).any(axis=1)

new_df_sa = df_sa[base_cols].copy()
new_df_sa['Flag_SupportiveAdultRelationships'] = mask.astype(int)  # 1 if condition met, 0 if not

# --- Overall Percentage ---
percent_flagged = new_df_sa['Flag_SupportiveAdultRelationships'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_SupportiveAdultRelationships = 1")

# --- Percentage by Gender ---
gender_pct = new_df_sa.groupby('GENDER')['Flag_SupportiveAdultRelationships'].mean() * 100
print("\n% by Gender:\n", gender_pct)

# --- Gender Counts + Percentages ---
gender_summary = new_df_sa.groupby('GENDER')['Flag_SupportiveAdultRelationships'].agg(['sum', 'count'])
gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print("\nGender Summary:\n", gender_summary)

# --- Percentage by Region ---
region_summary = new_df_sa.groupby('Region')['Flag_SupportiveAdultRelationships'].agg(['sum', 'count'])
region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print("\nRegion Summary:\n", region_summary)


96.58% of rows have Flag_SupportiveAdultRelationships = 1

% by Gender:
 GENDER
FEMALE    96.825397
MALE      96.402878
Name: Flag_SupportiveAdultRelationships, dtype: float64

Gender Summary:
          sum  count    percent
GENDER                        
FEMALE  2257   2331  96.825397
MALE    3216   3336  96.402878

Region Summary:
           sum  count    percent
Region                         
East     2120   2172  97.605893
Middle   1966   2037  96.514482
Unknown   220    255  86.274510
West     1167   1203  97.007481


### Housing_instability 

In [40]:

cols_to_keep = list(Housing_instability.keys())
df_hi = df_cans.loc[:, df_cans.columns.intersection(cols_to_keep)].rename(columns=Housing_instability).copy()

In [41]:
df_hi.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Housing,Housing Instability,Housing Instability,Housing Instability,Housing Instability,Region
0,353258,Madison,MALE,37040,0,1,0,0,0,West
1,706512,Sullivan,FEMALE,37642,0,0,0,0,0,East
2,1348374,Warren,FEMALE,37110,1,0,0,0,0,Middle
3,2470086,Hamilton,MALE,37218,2,1,0,1,2,East
4,3965628,Davidson,MALE,37725,0,0,0,0,0,Middle


In [44]:
print(df_hi.columns)


Index(['PERSON ID', 'RESPONSIBLE COUNTY', 'GENDER', 'PLACEMENT ZIP CODE',
       'Housing', 'Housing Instability', 'Housing Instability',
       'Housing Instability', 'Housing Instability', 'Region'],
      dtype='object')


In [48]:
hi_cols = df_hi.filter(regex=r'^Housing Instability$').columns
nums = df_hi[hi_cols].apply(pd.to_numeric, errors='coerce')  # strings → NaN
mask = nums.ge(2).any(axis=1)
df_hi_2 = df_hi[mask]


In [49]:
df_hi_2.head()

,PERSON ID,RESPONSIBLE COUNTY,GENDER,PLACEMENT ZIP CODE,Housing,Housing Instability,Housing Instability,Housing Instability,Housing Instability,Region
3,2470086,Hamilton,MALE,37218,2,1,0,1,2,East
5,3997434,Knox,MALE,30135,0,2,1,1,1,East
8,4451688,Hamilton,MALE,38401,0,2,0,3,0,East
11,4526460,Knox,MALE,37918,2,3,0,3,3,East
12,4578870,Rutherford,MALE,37185,1,2,1,3,0,Middle


In [50]:
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']
housing_cols = df_hi.filter(regex=r'^(Housing|Housing Instability)$').columns


In [51]:
nums = df_hi[housing_cols].apply(pd.to_numeric, errors='coerce')


In [52]:
mask = nums.ge(2).any(axis=1)

In [53]:
new_df = df_hi[base_cols].copy()
new_df['Flag_HousingIssue'] = mask.astype(int)  # 1 if condition met, 0 if not

In [54]:
new_df.head()

,PERSON ID,PLACEMENT ZIP CODE,Region,GENDER,Flag_HousingIssue
0,353258,37040,West,MALE,0
1,706512,37642,East,FEMALE,0
2,1348374,37110,Middle,FEMALE,0
3,2470086,37218,East,MALE,1
4,3965628,37725,Middle,MALE,0


In [58]:
percent_flagged = new_df['Flag_HousingIssue'].mean() * 100
print(f"{percent_flagged:.2f}% of rows have Flag_HousingIssue = 1")


54.39% of rows have Flag_HousingIssue = 1


In [ ]:
# Percentage of Flag_HousingIssue = 1 by gender
gender_pct = (
    new_df.groupby('GENDER')['Flag_HousingIssue']
          .mean() * 100
)
print(gender_pct)



GENDER
FEMALE    54.697555
MALE      54.166667
Name: Flag_HousingIssue, dtype: float64


In [62]:
gender_summary = (
    new_df.groupby('GENDER')['Flag_HousingIssue']
          .agg(['sum', 'count'])
)

gender_summary['percent'] = (gender_summary['sum'] / gender_summary['count']) * 100
print(gender_summary)


         sum  count    percent
GENDER                        
FEMALE  1275   2331  54.697555
MALE    1807   3336  54.166667


In [63]:
# Percentage of Flag_HousingIssue = 1 by Region
region_summary = (
    new_df.groupby('Region')['Flag_HousingIssue']
          .agg(['sum', 'count'])
)

region_summary['percent'] = (region_summary['sum'] / region_summary['count']) * 100
print(region_summary)


          sum  count    percent
Region                         
East     1159   2172  53.360958
Middle   1095   2037  53.755523
Unknown   111    255  43.529412
West      717   1203  59.600998


In [90]:
from functools import reduce

# List of all new dataframes you created
dfs_to_merge = [
    new_df_bic,
    new_df_cse,
    new_df_mer,
    new_df_pm,
    new_df_ptd,
    new_df_rs,
    new_df_sdc,
    new_df_tsr,
    new_df_sa,
    new_df
]

# Merge all on PERSON ID (outer join so no rows are dropped)
combined_df = reduce(
    lambda left, right: pd.merge(left, right, on=['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER'], how='outer'),
    dfs_to_merge
)

print("Combined master dataframe shape:", combined_df.shape)
combined_df.head()


Combined master dataframe shape: (5667, 14)


C:\Users\emros\AppData\Local\Temp\ipykernel_20368\2776637671.py:19: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Flag_MentalHealthIssue_x'} in the result is deprecated and will raise a MergeError in a future version.
  lambda left, right: pd.merge(left, right, on=['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER'], how='outer'),
C:\Users\emros\AppData\Local\Temp\ipykernel_20368\2776637671.py:19: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Flag_MentalHealthIssue_x'} in the result is deprecated and will raise a MergeError in a future version.
  lambda left, right: pd.merge(left, right, on=['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER'], how='outer'),
C:\Users\emros\AppData\Local\Temp\ipykernel_20368\2776637671.py:19: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Flag_MentalHealthIssue_x'} in the result is deprecated and will raise a MergeError in a future version.
  lambda left, right: pd.merge(left, right, on=['P

,PERSON ID,PLACEMENT ZIP CODE,Region,GENDER,Flag_MentalHealthIssue_x,Flag_MentalHealthIssue_y,Flag_MentalHealthIssue_x,Flag_MentalHealthIssue_y,Flag_MentalHealthIssue_x,Flag_MentalHealthIssue_y,Flag_MentalHealthIssue_x,Flag_MentalHealthIssue_y,Flag_SupportiveAdultRelationships,Flag_HousingIssue
0,353258,37040,West,MALE,0,0,0,0,0,0,0,0,1,0
1,706512,37642,East,FEMALE,0,0,0,0,0,0,0,0,1,0
2,1348374,37110,Middle,FEMALE,0,0,0,0,0,0,0,1,1,0
3,2470086,37218,East,MALE,1,1,1,0,0,1,0,0,1,1
4,3965628,37725,Middle,MALE,1,0,0,1,0,1,0,1,1,0


In [91]:
summary_rows = []

def add_summary(df, flag_col, name):
    total_pct = df[flag_col].mean() * 100
    gender_summary = (
        df.groupby('GENDER')[flag_col].agg(['sum', 'count'])
          .assign(percent=lambda x: (x['sum'] / x['count']) * 100)
    )
    region_summary = (
        df.groupby('Region')[flag_col].agg(['sum', 'count'])
          .assign(percent=lambda x: (x['sum'] / x['count']) * 100)
    )
    
    # Add overall summary row
    summary_rows.append({
        'Dataset': name,
        'Group': 'Overall',
        'GroupValue': 'All',
        'Sum': df[flag_col].sum(),
        'Count': len(df),
        'Percent': total_pct
    })
    
    # Add gender summary rows
    for idx, row in gender_summary.iterrows():
        summary_rows.append({
            'Dataset': name,
            'Group': 'Gender',
            'GroupValue': idx,
            'Sum': row['sum'],
            'Count': row['count'],
            'Percent': row['percent']
        })
    
    # Add region summary rows
    for idx, row in region_summary.iterrows():
        summary_rows.append({
            'Dataset': name,
            'Group': 'Region',
            'GroupValue': idx,
            'Sum': row['sum'],
            'Count': row['count'],
            'Percent': row['percent']
        })

# Call for each new dataframe you have:
add_summary(new_df_bic, 'Flag_MentalHealthIssue', 'BIC')
add_summary(new_df_cse, 'Flag_MentalHealthIssue', 'CSE')
add_summary(new_df_mer, 'Flag_MentalHealthIssue', 'MER')
add_summary(new_df_pm,  'Flag_MentalHealthIssue', 'PM')
add_summary(new_df_ptd, 'Flag_MentalHealthIssue', 'PTD')
add_summary(new_df_rs,  'Flag_MentalHealthIssue', 'RS')
add_summary(new_df_sdc, 'Flag_MentalHealthIssue', 'SDC')
add_summary(new_df_tsr, 'Flag_MentalHealthIssue', 'TSR')
add_summary(new_df_sa,  'Flag_SupportiveAdultRelationships', 'SA')
add_summary(new_df, 'Flag_HousingIssue', 'HI')

summary_df = pd.DataFrame(summary_rows)
summary_df.head(20)


,Dataset,Group,GroupValue,Sum,Count,Percent
0,BIC,Overall,All,3453.0,5667.0,60.931710
1,BIC,Gender,FEMALE,1185.0,2331.0,50.836551
2,BIC,Gender,MALE,2268.0,3336.0,67.985612
3,BIC,Region,East,1285.0,2172.0,59.162063
4,BIC,Region,Middle,1228.0,2037.0,60.284732
5,BIC,Region,Unknown,117.0,255.0,45.882353
6,BIC,Region,West,823.0,1203.0,68.412303
7,CSE,Overall,All,4073.0,5667.0,71.872243
8,CSE,Gender,FEMALE,1639.0,2331.0,70.313170
9,CSE,Gender,MALE,2434.0,3336.0,72.961631


In [92]:
# Save combined dataset with all flags
combined_df.to_csv("combined_flags_dataset.csv", index=False)

# Save summary table
summary_df.to_csv("flags_summary_table.csv", index=False)

print("✅ Both CSV files saved:")
print(" - combined_flags_dataset.csv")
print(" - flags_summary_table.csv")


✅ Both CSV files saved:
 - combined_flags_dataset.csv
 - flags_summary_table.csv


## Greater than 3

In [93]:
# ---------- HELPERS ----------
base_cols = ['PERSON ID', 'PLACEMENT ZIP CODE', 'Region', 'GENDER']

def build_flag_gt3(df, regex, flag_name):
    cols = df.filter(regex=regex).columns
    nums = df[cols].apply(pd.to_numeric, errors='coerce')
    mask = nums.gt(3).any(axis=1)  # strictly greater than 3
    subset = df[mask]
    new_df = df[base_cols].copy()
    new_df[flag_name] = mask.astype(int)
    return new_df, subset

In [94]:
def add_summary(df, flag_col, name, rows):
    total_pct = df[flag_col].mean() * 100
    rows.append({'Dataset': name, 'Group': 'Overall', 'GroupValue': 'All',
                 'Sum': int(df[flag_col].sum()), 'Count': int(len(df)), 'Percent': total_pct})
    g = (df.groupby('GENDER')[flag_col].agg(['sum','count'])
           .assign(percent=lambda x: (x['sum']/x['count'])*100))
    for idx, r in g.iterrows():
        rows.append({'Dataset': name, 'Group': 'Gender', 'GroupValue': idx,
                     'Sum': int(r['sum']), 'Count': int(r['count']), 'Percent': r['percent']})
    rgn = (df.groupby('Region')[flag_col].agg(['sum','count'])
             .assign(percent=lambda x: (x['sum']/x['count'])*100))
    for idx, r in rgn.iterrows():
        rows.append({'Dataset': name, 'Group': 'Region', 'GroupValue': idx,
                     'Sum': int(r['sum']), 'Count': int(r['count']), 'Percent': r['percent']})


In [100]:
# ---------- REBUILD FLAGS (>3) ----------
# Mental Health in multiple frames
new_df_bic_gt3, df_bic_2_gt3 = build_flag_gt3(df_bic, r'^Mental Health$', 'Flag_MentalHealthIssue_gt3')
new_df_cse_gt3, df_cse_2_gt3 = build_flag_gt3(df_cse, r'^Mental Health$', 'Flag_MentalHealthIssue_gt3')
new_df_mer_gt3, df_mer_2_gt3 = build_flag_gt3(df_mer, r'^Mental Health$', 'Flag_MentalHealthIssue_gt3')
new_df_pm_gt3,  df_pm_2_gt3  = build_flag_gt3(df_pm,  r'^Mental Health$', 'Flag_MentalHealthIssue_gt3')
new_df_ptd_gt3, df_ptd_2_gt3 = build_flag_gt3(df_ptd, r'^Mental Health$', 'Flag_MentalHealthIssue_gt3')
new_df_rs_gt3,  df_rs_2_gt3  = build_flag_gt3(df_rs,  r'^Mental Health$', 'Flag_MentalHealthIssue_gt3')
new_df_sdc_gt3, df_sdc_2_gt3 = build_flag_gt3(df_sdc, r'^Mental Health$', 'Flag_MentalHealthIssue_gt3')
new_df_tsr_gt3, df_tsr_2_gt3 = build_flag_gt3(df_tsr, r'^Mental Health$', 'Flag_MentalHealthIssue_gt3')
new_df_hi_gt3, df__hi_gt3 = build_flag_gt3(df_tsr, r'^(Housing, Housing_instability)$', 'Flag_HousingIssue_gt3')


In [101]:
# Supportive Adult Relationships in df_sa
new_df_sa_gt3, df_sa_2_gt3 = build_flag_gt3(df_sa, r'^Supportive Adult Relationships$', 'Flag_SupportiveAdultRelationships_gt3')

In [102]:
# ---------- COMBINE ALL (>3) ----------
from functools import reduce
dfs_to_merge_gt3 = [
    new_df_bic_gt3, new_df_cse_gt3, new_df_mer_gt3, new_df_pm_gt3,
    new_df_ptd_gt3, new_df_rs_gt3, new_df_sdc_gt3, new_df_tsr_gt3, new_df_sa_gt3, new_df_hi_gt3
]

In [103]:
combined_df_gt3 = reduce(
    lambda left, right: pd.merge(left, right, on=base_cols, how='outer'),
    dfs_to_merge_gt3
)

C:\Users\emros\AppData\Local\Temp\ipykernel_20368\952655046.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Flag_MentalHealthIssue_gt3_x'} in the result is deprecated and will raise a MergeError in a future version.
  lambda left, right: pd.merge(left, right, on=base_cols, how='outer'),
C:\Users\emros\AppData\Local\Temp\ipykernel_20368\952655046.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Flag_MentalHealthIssue_gt3_x'} in the result is deprecated and will raise a MergeError in a future version.
  lambda left, right: pd.merge(left, right, on=base_cols, how='outer'),
C:\Users\emros\AppData\Local\Temp\ipykernel_20368\952655046.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Flag_MentalHealthIssue_gt3_x'} in the result is deprecated and will raise a MergeError in a future version.
  lambda left, right: pd.merge(left, right, on=base_cols, how='outer'),


In [104]:
# ---------- SUMMARIES (>3) ----------
summary_rows_gt3 = []
add_summary(new_df_bic_gt3, 'Flag_MentalHealthIssue_gt3', 'BIC', summary_rows_gt3)
add_summary(new_df_cse_gt3, 'Flag_MentalHealthIssue_gt3', 'CSE', summary_rows_gt3)
add_summary(new_df_mer_gt3, 'Flag_MentalHealthIssue_gt3', 'MER', summary_rows_gt3)
add_summary(new_df_pm_gt3,  'Flag_MentalHealthIssue_gt3', 'PM',  summary_rows_gt3)
add_summary(new_df_ptd_gt3, 'Flag_MentalHealthIssue_gt3', 'PTD', summary_rows_gt3)
add_summary(new_df_rs_gt3,  'Flag_MentalHealthIssue_gt3', 'RS',  summary_rows_gt3)
add_summary(new_df_sdc_gt3, 'Flag_MentalHealthIssue_gt3', 'SDC', summary_rows_gt3)
add_summary(new_df_tsr_gt3, 'Flag_MentalHealthIssue_gt3', 'TSR', summary_rows_gt3)
add_summary(new_df_sa_gt3,  'Flag_SupportiveAdultRelationships_gt3', 'SA', summary_rows_gt3)
add_summary(new_df_hi_gt3, 'Flag_HousingIssue_gt3', 'HI', summary_rows_gt3)

summary_df_gt3 = pd.DataFrame(summary_rows_gt3)

In [105]:
# ---------- SAVE CSVs (>3) ----------
combined_df_gt3.to_csv("combined_flags_dataset_gt3.csv", index=False)
summary_df_gt3.to_csv("flags_summary_table_gt3.csv", index=False)

print("✅ Saved:")
print(" - combined_flags_dataset_gt3.csv")
print(" - flags_summary_table_gt3.csv")

✅ Saved:
 - combined_flags_dataset_gt3.csv
 - flags_summary_table_gt3.csv
